<a href="https://colab.research.google.com/github/arturowski/dw_matrix_car/blob/master/day3_simple_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
!pip install --upgrade tables
!pip install eli5

     |████████████████████████████████| 4.3MB 4.9MB/s 
  Found existing installation: tables 3.4.4
    Uninstalling tables-3.4.4:
      Successfully uninstalled tables-3.4.4
     |████████████████████████████████| 112kB 5.0MB/s 


In [0]:
import pandas as pd
import numpy as np
from sklearn.dummy import DummyRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import mean_absolute_error as mae
from sklearn.model_selection import cross_val_score
import eli5
from eli5.sklearn import PermutationImportance

##wczytywanie danych

In [9]:
!pwd

/content


In [10]:
cd "/content/drive/My Drive/Colab Notebooks/matrix/matrix_two/dw_matrix_car"

/content/drive/My Drive/Colab Notebooks/matrix/matrix_two/dw_matrix_car


In [11]:
ls

car.h5  data/  HelloGithub.ipynb  LICENSE  README.md


In [0]:
df = pd.read_hdf('car.h5')

##Dummy model

In [17]:
df.select_dtypes(np.number).columns

Index(['price_value', 'car_id'], dtype='object')

In [0]:
feats = ['car_id']
X = df[feats].values
y = df['price_value'].values

In [24]:
model = DummyRegressor()
model.fit(X,y)
y_pred = model.predict(X)
mae(y,y_pred)

39465.934630440985

In [26]:
df['price_currency'].value_counts()

PLN    106290
EUR       204
Name: price_currency, dtype: int64

In [44]:
df = df [df['price_currency'] != 'EUR']
df.shape

(106290, 155)

##Features

In [29]:
df['param_color'].factorize()[0]

array([-1, -1, -1, ..., -1, -1, -1])

In [0]:
df = df.drop(['breadcrumb', 'created_at', 'price_details', 'seller_address', 'seller_name'], axis = 1)

In [0]:
df = df.drop(['price_currency'], axis = 1)

In [40]:
df.head()


,price_value,seller_type,feature_czujniki-parkowania-przednie,feature_poduszka-powietrzna-chroniąca-kolana,feature_kurtyny-powietrzne,feature_klimatyzacja-dwustrefowa,feature_światła-led,feature_czujnik-zmierzchu,feature_elektrycznie-ustawiane-lusterka,feature_asr-(kontrola-trakcji),feature_poduszka-powietrzna-kierowcy,feature_cd,feature_elektryczne-szyby-przednie,feature_poduszka-powietrzna-pasażera,feature_system-start-stop,feature_światła-do-jazdy-dziennej,feature_komputer-pokładowy,feature_elektryczne-szyby-tylne,feature_klimatyzacja-manualna,feature_tapicerka-welurowa,feature_czujnik-deszczu,feature_światła-przeciwmgielne,feature_ogrzewanie-postojowe,feature_radio-niefabryczne,feature_regulowane-zawieszenie,feature_ogranicznik-prędkości,feature_zmieniarka-cd,feature_szyberdach,feature_isofix,feature_asystent-pasa-ruchu,feature_alufelgi,feature_bluetooth,feature_nawigacja-gps,feature_asystent-parkowania,feature_wspomaganie-kierownicy,feature_podgrzewana-przednia-szyba,feature_przyciemniane-szyby,feature_elektrycznie-ustawiane-fotele,feature_klimatyzacja-czterostrefowa,feature_tuner-tv,...,param_particle-filter,param_zarejestrowany-w-polsce,param_oferta-od,param_moc,param_make,param_kraj-pochodzenia,param_engine-power,param_year,param_serwisowany-w-aso,param_rodzaj-paliwa,param_możliwość-finansowania,param_registered-in-poland,param_transmission,param_miesięczna-rata,param_kod-silnika,param_version,param_opłata-początkowa,param_model-pojazdu,param_pojemność-skokowa,param_kierownica-po-prawej-(anglik),param_fuel-type,param_vat-free,param_acrylic,param_damaged,param_door-count,param_tuning,param_filtr-cząstek-stałych,param_nr-of-seats,param_kolor,param_engine-capacity,param_pearl,param_stan,param_wersja,param_emisja-co2,param_body-type,param_matowy,param_bezwypadkowy,param_akryl-(niemetalizowany),param_monthly-payment-value,car_id
0,45100.0,Dealer,False,False,False,False,False,False,False,True,False,False,False,False,False,False,True,True,True,False,False,True,False,False,False,False,False,False,False,False,False,True,False,False,True,False,False,False,False,False,...,None,None,Firmy,90 KM,None,None,None,None,None,Benzyna+LPG,None,None,None,None,None,None,None,Logan,898 cm3,None,None,None,None,None,None,None,None,None,Inny kolor,None,None,Nowe,II (2012-),None,None,None,Tak,None,None,0
2,28000.0,Osoba prywatna,False,False,False,True,False,False,True,True,True,True,True,True,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,True,False,False,False,False,False,...,None,Tak,Osoby prywatnej,115 KM,None,Polska,None,None,None,Diesel,None,None,None,None,None,None,None,3008,1 560 cm3,None,None,None,None,None,None,None,None,None,Czarny,None,None,Używane,I (2009-2016),None,None,None,Tak,None,None,2
3,229500.0,Dealer,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,False,False,True,True,True,False,True,True,True,True,True,False,True,True,True,True,True,True,True,True,False,True,...,None,None,Firmy,262 KM,None,Niemcy,None,None,Tak,Diesel,Tak,None,None,None,None,None,None,Cayenne,3 000 cm3,None,None,None,None,None,None,None,None,None,Czarny,None,None,Używane,II (2010-),None,None,None,None,None,None,3
4,19500.0,Osoba prywatna,False,False,False,False,False,True,True,True,True,True,True,True,False,False,True,True,False,False,True,True,False,False,False,False,False,False,True,False,True,False,False,False,True,False,False,False,True,False,...,None,Tak,Osoby prywatnej,110 KM,None,Polska,None,None,Tak,Diesel,None,None,None,None,None,None,None,308,1 560 cm3,None,None,None,None,None,None,None,None,None,Srebrny,None,None,Używane,T7 (2008-2013),None,None,None,Tak,None,None,4
5,164900.0,Dealer,True,True,True,False,True,True,True,True,True,True,True,True,False,True,True,True,False,False,True,False,False,False,False,False,False,True,True,True,True,True,True,False,True,True,True,False,False,False,...,None,Tak,Firmy,310 KM,None,Polska,None,None,Tak,Benzyna,No

In [0]:
for i in df.columns:
  if i != 'price_value':
    df[i] = df[i].factorize()[0]

In [49]:
df.head(50)

,price_value,seller_type,feature_czujniki-parkowania-przednie,feature_poduszka-powietrzna-chroniąca-kolana,feature_kurtyny-powietrzne,feature_klimatyzacja-dwustrefowa,feature_światła-led,feature_czujnik-zmierzchu,feature_elektrycznie-ustawiane-lusterka,feature_asr-(kontrola-trakcji),feature_poduszka-powietrzna-kierowcy,feature_cd,feature_elektryczne-szyby-przednie,feature_poduszka-powietrzna-pasażera,feature_system-start-stop,feature_światła-do-jazdy-dziennej,feature_komputer-pokładowy,feature_elektryczne-szyby-tylne,feature_klimatyzacja-manualna,feature_tapicerka-welurowa,feature_czujnik-deszczu,feature_światła-przeciwmgielne,feature_ogrzewanie-postojowe,feature_radio-niefabryczne,feature_regulowane-zawieszenie,feature_ogranicznik-prędkości,feature_zmieniarka-cd,feature_szyberdach,feature_isofix,feature_asystent-pasa-ruchu,feature_alufelgi,feature_bluetooth,feature_nawigacja-gps,feature_asystent-parkowania,feature_wspomaganie-kierownicy,feature_podgrzewana-przednia-szyba,feature_przyciemniane-szyby,feature_elektrycznie-ustawiane-fotele,feature_klimatyzacja-czterostrefowa,feature_tuner-tv,...,param_particle-filter,param_zarejestrowany-w-polsce,param_oferta-od,param_moc,param_make,param_kraj-pochodzenia,param_engine-power,param_year,param_serwisowany-w-aso,param_rodzaj-paliwa,param_możliwość-finansowania,param_registered-in-poland,param_transmission,param_miesięczna-rata,param_kod-silnika,param_version,param_opłata-początkowa,param_model-pojazdu,param_pojemność-skokowa,param_kierownica-po-prawej-(anglik),param_fuel-type,param_vat-free,param_acrylic,param_damaged,param_door-count,param_tuning,param_filtr-cząstek-stałych,param_nr-of-seats,param_kolor,param_engine-capacity,param_pearl,param_stan,param_wersja,param_emisja-co2,param_body-type,param_matowy,param_bezwypadkowy,param_akryl-(niemetalizowany),param_monthly-payment-value,car_id
0,45100.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,-1,-1,0,0,-1,-1,-1,-1,-1,0,-1,-1,-1,-1,-1,-1,-1,0,0,-1,-1,-1,-1,-1,-1,-1,-1,-1,0,-1,-1,0,0,-1,-1,-1,0,-1,-1,0
2,28000.0,1,0,0,0,1,0,0,1,0,1,1,1,1,0,0,0,1,1,0,0,1,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,...,-1,0,1,1,-1,0,-1,-1,-1,1,-1,-1,-1,-1,-1,-1,-1,1,1,-1,-1,-1,-1,-1,-1,-1,-1,-1,1,-1,-1,1,1,-1,-1,-1,0,-1,-1,1
3,229500.0,0,1,1,1,1,1,1,1,0,1,1,1,1,1,1,0,0,1,0,1,0,1,0,1,1,1,1,1,0,1,0,1,1,0,1,1,1,0,1,...,-1,-1,0,2,-1,1,-1,-1,0,1,0,-1,-1,-1,-1,-1,-1,2,2,-1,-1,-1,-1,-1,-1,-1,-1,-1,1,-1,-1,1,2,-1,-1,-1,-1,-1,-1,2
4,19500.0,1,0,0,0,0,0,1,1,0,1,1,1,1,0,0,0,0,1,0,1,0,0,0,0,0,0,0,1,0,1,1,0,0,0,0,0,0,1,0,...,-1,0,1,3,-1,0,-1,-1,0,1,-1,-1,-1,-1,-1,-1,-1,3,1,-1,-1,-1,-1,-1,-1,-1,-1,-1,2,-1,-1,1,3,-1,-1,-1,0,-1,-1,3
5,164900.0,0,1,1,1,0,1,1,1,0,1,1,1,1,0,1,0,0,1,0,1,1,0,0,0,0,0,1,1,1,1,0,1,0,0,1,1,0,0,0,...,-1,0,0,4,-1,0,-1,-1,0,2,-1,-1,-1,-1,-1,-1,-1,4,3,-1,-1,-1,-1,-1,-1,-1,-1,-1,3,-1,-1,1,4,-1,-1,-1,0,-1,-1,4
6,26900.0,0,0,0,1,0,0,0,1,1,1,1,1,1,0,0,1,1,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,...,-1,0,0,5,-1,0,-1,-1,-1,1,-1,-1,-1,-1,-1,-1,-1,5,4,-1,-1,-1,-1,-1,-1,-1,-1,-1,3,-1,-1,1,5,-1,-1,-1,0,-1,-1,5
7,124380.0,0,0,0,1,0,1,0,1,0,1,0,1,1,1,1,0,0,0,1,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,...,-1,-1,0,6,-1,-1,-1,-1,-1,1,0,-1,-1,-1,-1,-1,-1,6,5,-1,-1,-1,-1,-1,-1,-1,-1,-1,4,-1,-1,0,-1,-1,-1,-1,0,-1,-1,6
10,44999.0,0,1,0,1,1,0,1,1,0,1,1,1,1,0,0,0,0,1,0,1,0,0,0,0,1,0,0,1,0,1,0,1,0,0,0,1,1,0,0,...,-1,0,0,7,-1,0,-1,-1,0,1,0,-1,-1,-1,-1,-1,-1,7,6,-1,-1,-1,-1,-1,-1,-1,0,-1,2,-1,-1,1,6,-1,-1,-1,0,-1,-1,7
11,17500.0,1,0,0,0,0,0,0,1,1,1,1,1,1,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,...,-1,-1,1,8,-1,1,-1,-1,-1,2,-1,-1,-1,-1,-1,-1,-1,8,7,-1,-1,-1,-1,-1,-1,-1,-1,-1,1,-1,-1,1,-1,-1,-1,-1,-1,-1,-1,8
13,4000.0,1,0,0,0,1,0,0,1,1,1,0,1,1,0,1,0,1,1,1,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,...,-1,0,1,8,-1,1,-1,-1,-1,1,-1,-1,-1,-1,-1,-1,-1,9,8,-1,-1,-1,-1,-1,-1,-1,-1,-1,5,-1,-1,1,7,-1,-1,-1,0,-1,-1,9


In [0]:
y = df['price_value']

In [0]:
X = df.drop(['price_value'], axis = 1)

In [0]:
model = DecisionTreeRegressor(max_depth=5)

In [0]:
scores = cross_val_score(model, X, y, cv=3, scoring = 'neg_mean_absolute_error')

In [56]:
scores

array([-19625.57510982, -19438.56693715, -19635.62476514])

In [57]:
np.mean(scores)

-19566.588937368324

In [58]:
m = DecisionTreeRegressor(max_depth=5)
m.fit(X,y)

imp = PermutationImportance(m, random_state=0).fit(X,y)
eli5.show_weights(imp, feature_names=X.columns.values)

Weight,Feature
0.2544 ± 0.0064,param_napęd
0.1969 ± 0.0057,param_faktura-vat
0.1926 ± 0.0086,param_stan
0.1431 ± 0.0046,param_rok-produkcji
0.0619 ± 0.0053,param_moc
0.0419 ± 0.0005,feature_kamera-cofania
0.0418 ± 0.0015,param_skrzynia-biegów
0.0289 ± 0.0043,param_marka-pojazdu
0.0188 ± 0.0009,param_pojemność-skokowa
0.0164 ± 0.0002,feature_bluetooth
